In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os, glob
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import matplotlib.pyplot as plt

# =========================
# 0) 사용자가 바꿀 값
# =========================
NORMAL_FOLDER = "/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/00. Datasets/Kaggle(Car damage detection)/training/01-whole"   # ✅ 정상차량만 모아둔 폴더
MODEL_PATH    = "/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/JR/01.Model/damage_classifier_stage2(20epoch).pth"  # ✅ Stage2 저장 모델(state_dict)
BATCH_SIZE    = 64
NUM_WORKERS   = 2
IMG_SIZE      = 224

# 노트북 기준 라벨(중요!)
NORMAL_CLASS_IDX  = 0   # normal = 0
DAMAGED_CLASS_IDX = 1   # damaged = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# =========================
# 1) 노트북의 val/test transform 그대로
# =========================
val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])


# =========================
# 2) 폴더 이미지 Dataset (이미지 + 경로 반환)
# =========================
class FolderImageDataset(Dataset):
    def __init__(self, folder, transform=None):
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.webp", "*.JPEG", "*.JPG")
        paths = []
        for e in exts:
            paths.extend(glob.glob(os.path.join(folder, e)))
        self.paths = sorted(paths)
        if len(self.paths) == 0:
            raise ValueError(f"No images found in: {folder}")
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, path


# =========================
# 3) 노트북과 동일하게 ResNet18(2-class) 구성 후 state_dict 로드
#    (학습 때 pretrained=True였지만, state_dict 로드가 덮어쓰므로 weights=None로 생성해도 보통 문제 없습니다)
# =========================
def build_resnet18_2class():
    model = models.resnet18(weights=None)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, 2)
    return model

model = build_resnet18_2class()
state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()


# =========================
# 4) 추론 + "정상 폴더" 기준 정확도 계산
# =========================
ds = FolderImageDataset(NORMAL_FOLDER, transform=val_test_transform)
dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

total = 0
correct = 0
pred_normal = 0
pred_damaged = 0
mis_paths = []   # 정상인데 damaged로 예측된 이미지 목록(오답)

with torch.no_grad():
    for images, paths in dl:
        images = images.to(device)

        logits = model(images)                 # [B,2]
        probs  = torch.softmax(logits, dim=1)  # [B,2]
        preds  = torch.argmax(probs, dim=1)    # [B]

        preds = preds.cpu().numpy()

        for p, path in zip(preds, paths):
            total += 1
            if int(p) == NORMAL_CLASS_IDX:
                pred_normal += 1
                correct += 1
            else:
                pred_damaged += 1
                mis_paths.append(path)

acc = correct / total if total else 0.0

print("\n==== Normal-folder Evaluation (Ground truth: ALL NORMAL) ====")
print(f"Total images           : {total}")
print(f"Pred NORMAL (correct)  : {pred_normal}")
print(f"Pred DAMAGED (wrong)   : {pred_damaged}")
print(f"Accuracy               : {acc:.4f} ({correct}/{total})")

print("\n[Wrong samples] normal folder but predicted DAMAGED (show up to 30):")
for p in mis_paths[:30]:
    print(" -", p)


# =========================
# 5) (선택) 오답 이미지 몇 장 바로 보기
# =========================
def show_images(paths, max_show=16, cols=4, title="Misclassified (pred=damaged)"):
    if len(paths) == 0:
        print("No misclassified images to show.")
        return
    n = min(max_show, len(paths))
    rows = (n + cols - 1) // cols
    plt.figure(figsize=(4*cols, 4*rows))
    for i in range(n):
        img = Image.open(paths[i]).convert("RGB")
        plt.subplot(rows, cols, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(os.path.basename(paths[i]), fontsize=9)
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# 오답 이미지 확인
show_images(mis_paths, max_show=16, cols=4)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import os, glob
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import matplotlib.pyplot as plt

# =========================
# 0) 사용자가 바꿀 값
# =========================
DAMAGED_FOLDER = "/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/00. Datasets/Kaggle(Car damage detection)/training/00-damage"     # ✅ 파손차량만 모아둔 폴더
MODEL_PATH     = "/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK1_CAR_DETECTION/JR/01.Model/damage_classifier_stage2(20epoch).pth"  # ✅ Stage2 모델(state_dict)
BATCH_SIZE     = 64
NUM_WORKERS    = 2
IMG_SIZE       = 224

# 노트북 기준 라벨(중요!)
NORMAL_CLASS_IDX  = 0   # normal = 0
DAMAGED_CLASS_IDX = 1   # damaged = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# 1) 노트북의 val/test transform 그대로
# =========================
val_test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])

# =========================
# 2) 폴더 이미지 Dataset (이미지 + 경로 반환)
# =========================
class FolderImageDataset(Dataset):
    def __init__(self, folder, transform=None):
        exts = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.webp", "*.JPEG")
        paths = []
        for e in exts:
            paths.extend(glob.glob(os.path.join(folder, e)))
        self.paths = sorted(paths)
        if len(self.paths) == 0:
            raise ValueError(f"No images found in: {folder}")
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, path

# =========================
# 3) ResNet18(2-class) 구성 후 state_dict 로드
# =========================
def build_resnet18_2class():
    model = models.resnet18(weights=None)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, 2)
    return model

model = build_resnet18_2class()
state = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state)
model.to(device)
model.eval()

# =========================
# 4) 추론 + "파손 폴더" 기준 정확도 계산
#    - 정답은 전부 DAMAGED(1)
#    - 틀린 것: pred == NORMAL(0)  => (FN 개념)
# =========================
ds = FolderImageDataset(DAMAGED_FOLDER, transform=val_test_transform)
dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

total = 0
correct = 0
pred_normal = 0
pred_damaged = 0

wrong_paths = []   # 파손 폴더인데 normal로 예측된 이미지 목록 (=오답/FN)

with torch.no_grad():
    for images, paths in dl:
        images = images.to(device)

        logits = model(images)                 # [B,2]
        probs  = torch.softmax(logits, dim=1)  # [B,2]
        preds  = torch.argmax(probs, dim=1)    # [B]

        preds = preds.cpu().numpy()
        for p, path in zip(preds, paths):
            total += 1
            if int(p) == DAMAGED_CLASS_IDX:
                pred_damaged += 1
                correct += 1
            else:
                pred_normal += 1
                wrong_paths.append(path)

acc = correct / total if total else 0.0

print("\n==== Damaged-folder Evaluation (Ground truth: ALL DAMAGED) ====")
print(f"Total images           : {total}")
print(f"Pred DAMAGED (correct) : {pred_damaged}")
print(f"Pred NORMAL (wrong/FN) : {pred_normal}")
print(f"Accuracy               : {acc:.4f} ({correct}/{total})")

print("\n[Wrong samples] damaged folder but predicted NORMAL (show up to 30):")
for p in wrong_paths[:30]:
    print(" -", p)

# =========================
# 5) (선택) 오답 이미지 몇 장 바로 보기
# =========================
def show_images(paths, max_show=16, cols=4, title="Wrong (pred=normal) from damaged folder"):
    if len(paths) == 0:
        print("No wrong images to show.")
        return
    n = min(max_show, len(paths))
    rows = (n + cols - 1) // cols
    plt.figure(figsize=(4*cols, 4*rows))
    for i in range(n):
        img = Image.open(paths[i]).convert("RGB")
        plt.subplot(rows, cols, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(os.path.basename(paths[i]), fontsize=9)
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

show_images(wrong_paths, max_show=16, cols=4)


Output hidden; open in https://colab.research.google.com to view.